<a href="https://colab.research.google.com/github/takatakamanbou/AdvML/blob/2024/AdvML2024_ex07notebookA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AdvML ex07notebookA

<img width=72 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/AdvML/AdvML-logo.png"> [この授業のウェブページ](https://www-tlab.math.ryukoku.ac.jp/wiki/?AdvML)




----
## 準備
----


「例1」のセクションの実行のために必要な import 文等は，そちらのせくよんの中の「準備」にまとめてあります．

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# OpenCV のあれこれ
import cv2
from google.colab.patches import cv2_imshow

---
## 畳み込みニューラルネットワーク
---


**畳み込み** (convolution) という演算を行う層（**畳み込み層**）を含むニューラルネットのこと．
Convolutional Neural Network を略して CNN と呼ばれる．

---
### 画像のような情報を階層型ニューラルネットで扱おう

#### 問題点と改善策

#### 1次元の畳み込み

$T$ 個の数値 $x_1, x_2, \ldots, x_T$ がこの順に一列に並んだデータを考える．
このとき，このデータに対する1次元畳み込みを次式で定義する（注1）．

$$
y_t = \sum_{k=0}^{K-1} w_{k} x_{t+k}
$$

$y_t$ の値は，$K$ 個の値 $x_t, x_{t+1}, \ldots, x_{t+K-1}$  に $w_{0}, w_{1}, \ldots, w_{K-1}$ のそれぞれを掛けてそれらの和をとったものである（注2）．
$w_k (k = 0, 1, \ldots, K-1)$ という $K$ 個の値の組を，「カーネル」または「フィルタ」という．
カーネルの値を変えると，$x_t$ が同じ値でも得られる $y_t$ の値は変化する．

※ 注1 ここでは，ニューラルネットの分野で一般的な「畳み込み」の定義を採用している．
この定義は，「畳み込み」という語が元々用いられてきた信号処理の分野における定義とは少し異なっているので，注意が必要である．この定義は，信号処理の世界では「相互相関」と呼ばれるものに近い．

※ 注2 この式の通りだと，$t = T - K + 1, \ldots, T$ の場合に $x_t$ の添え字が $T$ より大きくなる場合が生じるため，$y_t$ を計算できなくなる．
畳み込みニューラルネットにおいては，範囲からはみ出した部分に $0$ が入っていると仮定して計算する，等の処理（パディングと呼ばれる）がよく行われる．




#### 2次元の畳み込み

$W \times H$ 個の数値 $v_{x, y}$ ($x = 1, 2, \ldots, W, y = 1, 2, \ldots, H$) が縦横にならんだデータを考える（例: グレイスケール画像）．
このデータに対する2次元畳み込みを次式で定義する（注3）．

$$
y_{x, y} = \sum_{i = 0}^{K-1}\sum_{j=0}^{K-1} w_{i, j} v_{x+i, y+j}
$$

1次元の場合と同様に，$w_{i, j}$ ($i = 0, 1, \ldots, K-1, j = 0, 1, \ldots, K-1$) という $K^2$ 個の値の組を「カーネル」または「フィルタ」という

※ 注3 ここではカーネルの縦横のサイズが同じと仮定している．これは説明の簡単のためであり，実際には縦横でサイズが異なっていてもよい．
また，画像の場合，$y_{x,y}$ の値は，カーネルの中心が $(x, y)$ に来るようにして計算する方が都合がよいので，画像処理や畳み込みニューラルネットではそのような実装となっていることが多い．

#### 3次元畳み込み

カラー画像は，縦横に加えて色の軸をもつ3次元配列と考えることができるので，
画像を扱う畳み込みニューラルネットでは 3次元の畳み込みを用いるのが一般的である．
2次元の場合の自然な拡張として容易に理解できるので，式の定義や説明は省略する．

#### グレイスケール画像の畳み込み（フィルタリング）

畳み込みがどのような計算であるかを直感的に理解するために，グレイスケール画像に2次元畳み込みを適用する実験をやってみよう．

In [ ]:
# 実験用画像を入手して表示する

! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/hogelogo.png
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/uni3.png

imgHOGE = cv2.imread('hogelogo.png', cv2.IMREAD_GRAYSCALE)
print(imgHOGE.shape)
cv2_imshow(imgHOGE)

print()

imgUni3G = cv2.imread('uni3.png', cv2.IMREAD_GRAYSCALE)
print(imgUni3G.shape)

cv2_imshow(imgUni3G)

まずは，「平滑化」呼ばれる処理の例． 画像がぼける．
`kx`, `ky`でカーネルの幅と高さを指定しているので，変えてみよう．

In [ ]:
# 平滑化フィルタ
kx, ky = 3, 3
kernel = np.ones((ky, kx)) / (kx*ky)
print(kernel)

# フィルタリング
img1 = cv2.filter2D(imgHOGE, cv2.CV_32F, kernel)
img2 = cv2.filter2D(imgUni3G, cv2.CV_32F, kernel)

# 表示
cv2_imshow(img1)
print()
cv2_imshow(img2)

次は，Sobel フィルタと呼ばれるフィルタの例．この例の Sobel フィルタは，画像の水平方向のエッジ（明るさが急激に変化する箇所）に反応する．
この場合，出力が正負の値をとり，元の画素値の範囲におさまらなくなるので，0 が画素値128の灰色，正の所が白，負の所が黒になるように値を変換している．

In [ ]:
# 水平方向のエッジを検出する Sobel フィルタ
kernel = np.array( [[-1, 0, 1],
                   [-2, 0, 2],
                   [-1, 0, 1]] )
print(kernel)

# フィルタリング
img1 = cv2.filter2D(imgHOGE, cv2.CV_32F, kernel)
img2 = cv2.filter2D(imgUni3G, cv2.CV_32F, kernel)

# フィルタリング後の値が [0, 255] の範囲におさまるようにする
img1 = img1 / np.max(np.abs(img1)) * 127 + 128
img2 = img2 / np.max(np.abs(img2)) * 127 + 128

# 表示
cv2_imshow(img1)
print()
cv2_imshow(img2)

Sobel フィルタの向きを変えると，垂直方向のエッジも抽出できる．

In [ ]:
# 垂直方向のエッジを検出する Sobel フィルタ
kernel = np.array( [[-1, -2, -1],
                    [ 0,  0,  0],
                    [ 1,  2,  1]] )
print(kernel)

# フィルタリング
img1 = cv2.filter2D(imgHOGE, cv2.CV_32F, kernel)
img2 = cv2.filter2D(imgUni3G, cv2.CV_32F, kernel)

# フィルタリング後の値が [0, 255] の範囲におさまるようにする
img1 = img1 / np.max(np.abs(img1)) * 127 + 128
img2 = img2 / np.max(np.abs(img2)) * 127 + 128

# 表示
cv2_imshow(img1)
print()
cv2_imshow(img2)

----
### 畳み込みニューラルネットの構造

典型的な畳み込みニューラルネット（特に画像識別に適用されるもの）は，入力側から順に， **畳み込み層** と **プーリング層** (pooling layer) を何層か繰り返した後に **全結合層** (fully connected layer) を何層か繰り返す，という構造をしている．

#### 畳み込み層

#### プーリング層

#### 全結合層

---
### 例1: 手書き数字を識別する CNN

MNISTデータセット（の一部）を用いた実験をやってみよう．レポート課題でも同じデータセットを用いていた．

#### 準備

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

# scikie-learn のあれこれ
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split

# PyTorch 関係のほげ
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor
import torchsummary

In [ ]:
# MNIST データセットの入手
Xraw, yraw = fetch_openml('mnist_784', version=1, parser='auto', return_X_y=True, as_frame=False)
Xall = Xraw[:20000] / 255.0     # 画素値が [0, 255] の整数値なので [0, 1] の浮動小数点数値に変換
yall = yraw[:20000].astype(int) # クラスラベル．0 から 9 の整数値

# 学習データとテストデータの分割
XL, XT, yL, yT = train_test_split(Xall, yall, test_size=4000, random_state=4649, stratify=yall)
print(XL.shape, yL.shape)
print(XT.shape, yT.shape)

#### PyTorch

ここでは，Python の深層学習フレームワーク（Python で深層学習のプログラムを作成するためのソフトウェアライブラリ）の代表格である PyTorch を用いる．

https://pytorch.org/

PyTorch のプログラムは，通常通り CPU 上で実行するだけでなく，GPU 上で実行することもできる．
GPU は画像処理等の計算に特化している分，CPU よりも並列計算の性能が優れている．
ニューラルネットの計算は並列処理に向いているため， GPU を利用することで，より高速な計算が可能となる．

Google Colab の場合，デフォルトでは PyTorch のプログラムも CPU 上で実行されるが，次のように設定することで，GPU を利用できるようになる．

1. Colab のメニューから「ランタイム」>「ランタイムのタイプを変更」>「CPU」に代えて「T4 GPU」を選択して「保存」
1. ランタイムが初期化されるので，全てのコードセルを最初から実行し直す必要ある．
このセクションの実験の場合，「例1」のセクションの「準備」のコードセルを実行し直すのでok
1. 次のコードセルを実行して，`cuda` と表示されれば， PyTorch で GPU が使える状態になっている．`cpu` と表示される場合は，GPU が使えるようになっておらず，PyTorch のプログラムは CPU で実行される．


In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

#### 実験の準備

実験で用いるプログラムの準備

In [ ]:
# データを扱うためのクラス
#
class MMDataset(Dataset):

    def __init__(self, dataX, dataY):
        self.X = dataX.reshape((-1, 1, 28, 28))
        self.Y = dataY

    def __len__(self):
        return len(self.Y)

    def __getitem__(self, idx):
        X = torch.tensor(self.X[idx], dtype=torch.float32)
        y = torch.tensor(self.Y[idx], dtype=torch.int64)
        return X, y

In [ ]:
# 階層型ニューラルネットの構造と入出力を定義するクラス
#
class MLP(nn.Module):

    def __init__(self, numNeurons):
        super(MLP, self).__init__()
        numLayers = len(numNeurons)
        assert numLayers >= 2
        L = [nn.Flatten()] # [1, 28, 28] => 784
        # 中間層
        for i in range(numLayers-2):
            L.append(nn.Linear(numNeurons[i], numNeurons[i+1]))
            L.append(nn.ReLU())
        # 出力層
        L.append(nn.Linear(numNeurons[-2], numNeurons[-1]))
        self.layers = nn.ModuleList(L)
        self.numNeurons = numNeurons

    def forward(self, X):
        for layer in self.layers:
            X = layer(X)
        return X

In [ ]:
# 畳み込みニューラルネットの構造と入出力を定義するクラス
#
class CNN(nn.Module):

    def __init__(self):
        super(CNN, self).__init__()
        L = []
        # 畳み込み層1
        L.append(nn.Conv2d(1, 16, 5)) # output: [16, 24, 24]
        L.append(nn.ReLU())
        # 畳み込み層2
        L.append(nn.Conv2d(16, 32, 3))  # output: [32, 22, 22]
        L.append(nn.ReLU())
        # プーリング層
        L.append(nn.MaxPool2d(2)) # output: [32, 11, 11]
        # 全結合層1
        L.append(nn.Flatten()) # 32x11x11 = 3872
        L.append(nn.Linear(3872, 400))
        # 全結合層2（出力層）
        L.append(nn.Linear(400, 10))
        self.layers = nn.ModuleList(L)

    def forward(self, X):
        for layer in self.layers:
            X = layer(X)
        return X

In [ ]:
# 学習の関数
#
def train(model, lossFunc, optimizer, dl):
    loss_sum = 0.0
    ncorrect = 0
    n = 0
    for i, (X, lab) in enumerate(dl):
        X, lab = X.to(device), lab.to(device)
        Y = model(X)           # 一つのバッチ X を入力して出力 Y を計算
        loss = lossFunc(Y, lab) # 正解ラベル lab に対する loss を計算
        optimizer.zero_grad()   # 勾配をリセット
        loss.backward()         # 誤差逆伝播でパラメータ更新量を計算
        optimizer.step()         # パラメータを更新
        n += len(X)
        loss_sum += loss.item()  # 損失関数の値
        ncorrect += (Y.argmax(dim=1) == lab).sum().item()  # 正解数

    return loss_sum/n, ncorrect/n

In [ ]:
# 損失関数や識別率の値を求める関数
#
@torch.no_grad()
def evaluate(model, lossFunc, dl):
    loss_sum = 0.0
    ncorrect = 0
    n = 0
    for i, (X, lab) in enumerate(dl):
        X, lab = X.to(device), lab.to(device)
        Y = model(X)           # 一つのバッチ X を入力して出力 Y を計算
        loss = lossFunc(Y, lab)  # 正解ラベル lab に対する loss を計算
        n += len(X)
        loss_sum += loss.item() # 損失関数の値
        ncorrect += (Y.argmax(dim=1) == lab).sum().item()  # 正解数

    return loss_sum/n, ncorrect/n

#### 実験

次のコードセルでは，データ読み込みの処理を行う仕組みを用意する．

In [ ]:
# データ読み込みの仕組みを作る
dsL = MMDataset(XL, yL)
dsT = MMDataset(XT, yT)
dlL = DataLoader(dsL, batch_size=100, shuffle=True)
dlT = DataLoader(dsT, batch_size=100, shuffle=False)

次のコードセルを実行すると，ニューラルネットモデルが作られ， `net` という変数がそれを指す．

```
net = ...
```

の行のコメントの付け方を変えれば，モデルの構造を変えることができる．

In [ ]:
# ネットワークモデルの定義
net = MLP([784, 10]).to(device)                # ロジスティック回帰
#net = MLP([784, 1000, 10]).to(device)         #2層（中間層1層）の階層型ニューラルネット
#net = MLP([784, 1000, 1000, 10]).to(device)  #3層（中間層2層）の階層型ニューラルネット
#net = CNN().to(device)                          # 畳み込みニューラルネット

# ネットワークの構造を表示
torchsummary.summary(net, (1, 28, 28))

# 損失関数（交差エントロピー） とパラメータ最適化器の設定
loss_func = nn.CrossEntropyLoss(reduction='sum')
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)

次のコードセルを実行すると，学習とテストが行われる．

【参考】 Google Colab （無料ユーザ） での実行時間
- CPU
    - `net = MLP([784, 1000, 10]).to(device)`: 2分
    - `net = MLP([784, 1000, 1000, 10]).to(device)`: 3分
    - `net = CNN().to(device)`: 9分
- GPU (T4 GPU)
    - `net = MLP([784, 1000, 10]).to(device)`: 24秒
    - `net = MLP([784, 1000, 1000, 10]).to(device)`: 26秒
    - `net = CNN().to(device)`: 31秒


In [ ]:
# 学習の繰り返し回数
nepoch = 30

# 学習
L = []
print(f'学習データ数: {len(dsL)}  テストデータ数: {len(dsT)}')
print()
print('# epoch  lossL  lossT  rateL  rateT')
for t in range(1, nepoch+1):
    lossL, rateL = train(net, loss_func, optimizer, dlL)
    lossT, rateT = evaluate(net, loss_func, dlT)
    L.append([t, lossL, lossT, rateL, rateT])
    if (t < 10) or (t % 10 == 0):
        print(f'{t}   {lossL:.5f}   {lossT:.5f}   {rateL:.4f}   {rateT:.4f}')

# 学習曲線の表示
data = np.array(L)
fig, ax = plt.subplots(1, 2, facecolor='white', figsize=(12, 4))
ax[0].plot(data[:, 0], data[:, 1], '.-', label='loss for training data')
ax[0].plot(data[:, 0], data[:, 2], '.-', label='loss for test data')
ax[0].axhline(0.0, color='gray')
ax[0].set_ylim(-0.05, 1.0)
ax[0].legend()
ax[0].set_title(f'loss')
ax[1].plot(data[:, 0], data[:, 3], '.-', label='accuracy for training data')
ax[1].plot(data[:, 0], data[:, 4], '.-', label='accuracy for test data')
ax[1].axhline(1.0, color='gray')
ax[1].set_ylim(0.8, 1.025)
ax[1].legend()
ax[1].set_title(f'accuracy')
plt.show()

# 学習後の損失と識別率
loss2, rrate = evaluate(net, loss_func, dlL)
print(f'# lossL: {loss2:.5f}  rateL: {rrate:.4f}', end='   ')
loss2, rrate = evaluate(net, loss_func, dlT)
print(f'# lossT: {loss2:.5f}  rateT: {rrate:.4f}')

---
### 例2: VGG16

次のリンク先に，2023年度「機械学習I/II」第15回の notebook がある．

https://github.com/takatakamanbou/ML/blob/2023/ex15notebookC.ipynb

この notebook を入手して，冒頭の「#準備あれこれ」のコードセルを実行してから，「1000種類の物体を識別するニューラルネットを動かしてみよう」の部分を読んで実行しよう（とりあえず「実験その1」まで）．


「実験その1」を実行できたら，その下に次の内容のコードセルを追加して実行してみよう．
VGG16 のネットワークの構造を表示させることができる．

```
import torchsummary
torchsummary.summary(vgg16, (3, 224, 224))
```

「実験その2」，「実験その3」もどうぞ．